<a href="https://colab.research.google.com/github/spinafex/crispy-happiness/blob/main/SignatureOfALine_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preprocess the data: here we calculate the slope and intercept which can be used to generate a hedge ratio

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats


def calculate_spread(filename):
    data = pd.read_csv(filename)

    # Check if specified columns exist
    if 'CO1' not in data.columns or 'CL1' not in data.columns:
        raise ValueError(f"Columns 'CO1' and 'CL1' not found in CSV.")

    # Handle missing values by dropping rows with missing data
    data.dropna(subset=['CO1', 'CL1'], inplace=True)  # Drop rows with NaNs in either column

    # Convert pandas Series to NumPy arrays
    d_ts1 = data['CO1'].to_numpy()
    d_ts2 = data['CL1'].to_numpy()

    # Ensure correct shape (no need for adjustments since data has no NaNs)
    d_ts1 = np.diff(d_ts1)
    d_ts2 = np.diff(d_ts2)

    #slope, intercept = np.linalg.lstsq(d_ts1[:, np.newaxis], d_ts2, rcond=None)[0]
    slope, intercept, _, _, _ = stats.linregress(d_ts1, d_ts2)

    #return d_ts1, d_ts2
    return slope, intercept

filename = 'commodities.csv'
slope, intercept = calculate_spread(filename)
print(f"{slope}, {intercept}")

0.9024122927513858, 0.004363431571753323


Here we calculate the hedge ratio directly. The issues are the following a) the hedge ratio may not be constant and may need to be adjusted which could require a Kalman filter or some other technique, b) we are returning the spread between the differenced series (not the prices) which I think is more likely to be a OU series, c) here, we drop all rows where either asset1 or asset2 does not have data (we do not try to forward fill) which will have implications for the differenced series. Leaving them in and forward filling will have other implications so there is no perfect solution but may want to return to this at a later date to check what works best.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats


def calculate_spread(filename):
    data = pd.read_csv(filename)

    # Check if specified columns exist
    if 'CO1' not in data.columns or 'CL1' not in data.columns:
        raise ValueError(f"Columns 'CO1' and 'CL1' not found in CSV.")

    # Handle missing values by dropping rows with missing data
    data.dropna(subset=['CO1', 'CL1'], inplace=True)  # Drop rows with NaNs in either column

    # Convert pandas Series to NumPy arrays
    d_ts1 = data['CO1'].to_numpy()
    d_ts2 = data['CL1'].to_numpy()

    # Ensure correct shape (no need for adjustments since data has no NaNs)
    d_ts1 = np.diff(d_ts1)
    d_ts2 = np.diff(d_ts2)

    #slope, intercept = np.linalg.lstsq(d_ts1[:, np.newaxis], d_ts2, rcond=None)[0]
    slope, intercept, _, _, _ = stats.linregress(d_ts1, d_ts2)

    x = d_ts2 - slope * d_ts1
    #return d_ts1, d_ts2
    #return slope, intercept
    return x

filename = 'commodities.csv'
x = calculate_spread(filename)
#print(f"{slope}, {intercept}")
print(x)

[ 0.29926535 -0.27583334  0.24949562 ...  0.34562499 -0.21563595
  0.06196271]


# 1. Compute log signatures

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import esig.tosig as ts

def calculate_spread(filename):
    data = pd.read_csv(filename)

    # Check if specified columns exist
    if 'CO1' not in data.columns or 'CL1' not in data.columns:
        raise ValueError(f"Columns 'CO1' and 'CL1' not found in CSV.")

    # Handle missing values by dropping rows with missing data
    data.dropna(subset=['CO1', 'CL1'], inplace=True)  # Drop rows with NaNs in either column

    # Convert pandas Series to NumPy arrays
    d_ts1 = data['CO1'].to_numpy()
    d_ts2 = data['CL1'].to_numpy()

    # Ensure correct shape (no need for adjustments since data has no NaNs)
    d_ts1 = np.diff(d_ts1)
    d_ts2 = np.diff(d_ts2)

    #slope, intercept = np.linalg.lstsq(d_ts1[:, np.newaxis], d_ts2, rcond=None)[0]
    slope, intercept, _, _, _ = stats.linregress(d_ts1, d_ts2)

    x = d_ts2 - slope * d_ts1

    # Compute augmented truncated (n=3) log signature
    signature = ts.stream2sig(x, 3)

    return signature

# Example usage
filename = 'commodities.csv'
spread_signature = calculate_spread(filename)
print(spread_signature.shape)


(585,)


I do not think that this script is calculating the augmented truncated log-signature correctly

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats


def calculate_spread(filename, window=10, tau=1):
    data = pd.read_csv(filename)

    # Check if specified columns exist
    if 'CO1' not in data.columns or 'CL1' not in data.columns:
        raise ValueError(f"Columns 'CO1' and 'CL1' not found in CSV.")

    # Handle missing values by dropping rows with missing data
    data.dropna(subset=['CO1', 'CL1'], inplace=True)  # Drop rows with NaNs in either column

    # Convert pandas Series to NumPy arrays
    d_ts1 = data['CO1'].to_numpy()
    d_ts2 = data['CL1'].to_numpy()

    # Ensure correct shape (no need for adjustments since data has no NaNs)
    d_ts1 = np.diff(d_ts1)
    d_ts2 = np.diff(d_ts2)

    # Handle potential issues with differenced data length (optional)
    if len(d_ts1) != len(d_ts2):
        print("Warning: Differenced series have different lengths. Check data integrity.")

    # Perform linear regression using linregress
    slope, intercept, _, _, _ = stats.linregress(d_ts1, d_ts2)

    # Calculate spread based on fitted line
    x = d_ts2 - slope * d_ts1

    # Truncated, augmented log signatures
    log_sig = np.log(np.abs(x) + 1e-8)  # Add small epsilon to avoid log(0)
    truncated_log_sig = log_sig[-window:]  # Truncate to last 'window' elements
    augmented_log_sig = np.append(truncated_log_sig, tau * np.diff(truncated_log_sig))

    return augmented_log_sig


# Example usage
filename = 'commodities.csv'
window = 15  # Adjust window size as needed
tau = 0.5  # Adjust time constant as needed
augmented_log_sig = calculate_spread(filename, window, tau)

if augmented_log_sig is not None:
    print("Truncated, augmented log signatures:")
    print(augmented_log_sig)
else:
    print("Spread calculation failed.")


Truncated, augmented log signatures:
[-2.01449176 -1.81903591 -1.48128005 -1.54788491  0.05700617 -1.35652262
 -1.61300787 -3.32125574 -2.92975542 -1.46539451 -7.23113662 -4.11226757
 -1.0624009  -1.53416365 -2.7812223   0.09772792  0.16887793 -0.03330243
  0.80244554 -0.7067644  -0.12824262 -0.85412394  0.19575016  0.73218045
 -2.88287106  1.55943453  1.52493334 -0.23588138 -0.62352932]


In [ ]:
import numpy as np
from scipy.stats import norm
import torch
import torch.nn as nn
import torch.optim as optim

# Parameters
n_samples = 1000
n_timesteps = 100
truncation_level_N = 3  # Truncation level for log-signature
n_epochs = 1000
lr = 0.01
num_dimensions = 3

# Generate time series data
mu = 0.1
sigma = 0.2
dt = 1 / n_timesteps
X0 = np.zeros((n_samples, 2))

# Discretized OU process
X = np.zeros((n_samples, n_timesteps + 1, 2))
X[:, 0, :] = X0
for i in range(n_samples):
    for t in range(n_timesteps):
        X[i, t + 1, :] = X[i, t, :] + mu * (1 - X[i, t, :]) * dt + sigma * np.sqrt(dt) * np.random.randn(2)

# Payoff process (long first asset, short second asset)
Y = X[:, :, 0] - X[:, :, 1]

# Compute truncated log-signatures
time = np.linspace(0, 1, n_timesteps + 1)
time = np.repeat(time[None, :], n_samples, axis=0)[:, :, None]
augmented_X = np.concatenate((time, X), axis=2)

log_signatures = []
for i in range(n_samples):
    path = augmented_X[i]
    log_sig = []
    for j in range(n_timesteps + 1):
        increment = path[j] - path[j - 1] if j > 0 else path[0]
        num_dimensions = len(increment)

        # Initialize log_sig_n with the correct size
        if truncation_level_N == 1:
            log_sig_n = np.zeros(num_dimensions + 1)
        else:
            size = int((num_dimensions ** (truncation_level_N + 1) - 1) / (num_dimensions - 1))
            log_sig_n = np.zeros(size)
        log_sig_n[0] = 1.0  # Initialize with 1 (empty word)

        if truncation_level_N > 1:
            for n in range(1, truncation_level_N):
                start_idx = int((num_dimensions ** n - 1) / (num_dimensions - 1))
                end_idx = int((num_dimensions ** (n + 1) - 1) / (num_dimensions - 1))
                if n == 1:
                    log_sig_n[start_idx + 1:start_idx + num_dimensions + 1] = increment
                else:
                    outer_product = np.multiply.outer(log_sig_n[:start_idx + 1], increment).ravel() / (n + 1)
                    log_sig_n[start_idx + 1:end_idx + 1] = outer_product[:end_idx - start_idx]
        else:
            log_sig_n[1:num_dimensions + 1] = increment

        log_sig.append(log_sig_n)
    log_signatures.append(np.array(log_sig))
log_signatures = np.array(log_signatures)

#print(log_signatures)


# 2. Optimizing loss function with neural network

In [ ]:
import numpy as np
from scipy.stats import norm
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network
class LogSignatureNet(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super(LogSignatureNet, self).__init__()
        layers = []
        last_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(last_dim, hidden_dim))
            layers.append(nn.ReLU())
            last_dim = hidden_dim
        layers.append(nn.Linear(last_dim, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Parameters
n_samples = 1000
n_timesteps = 100
truncation_level_N = 3
n_epochs = 10 #00
lr = 0.01

# Generate dummy data for truncated log-signatures
log_signatures = np.random.randn(n_samples, n_timesteps + 1, int(truncation_level_N * (3 ** truncation_level_N - 1) / 2))

# Generate dummy payoff process
Y = np.random.randn(n_samples, n_timesteps + 1)

# Define the loss function
def loss_fn(net):
    loss = 0
    for m in range(n_samples):
        y0 = Y[m, 0]
        integral = 0
        for j in range(n_timesteps):
            input_tensor = torch.tensor(log_signatures[m, j + 1], dtype=torch.float32).unsqueeze(0)
            dnn_output = net(input_tensor).item()
            integral += (1 - norm.cdf(integral)) * (Y[m, j + 1] - Y[m, j])
        loss += -(y0 + integral)
    return loss / n_samples

# Optimize the loss function
input_dim = log_signatures.shape[-1]
hidden_dims = [5, 30, 30]  # Example architecture
net = LogSignatureNet(input_dim, hidden_dims)
optimizer = optim.Adam(net.parameters(), lr=lr)
for epoch in range(n_epochs):
    optimizer.zero_grad()
    loss = loss_fn(net)
    loss_tensor = torch.tensor(loss, requires_grad=True)
    loss_tensor.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

Epoch 0, Loss: -1.817154


# 3. Generate sample paths from an OU process and use these to generate a learned policy

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Parameters
n_samples = 750
n_timesteps = 75
truncation_level_N = 3
hidden_dims = [32, 64, 32]
max_epochs = 500
lr = 0.001

# Define the neural network architecture
class LogSignatureNet(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super(LogSignatureNet, self).__init__()
        layers = []
        last_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(last_dim, hidden_dim))
            layers.append(nn.ReLU())
            last_dim = hidden_dim
        layers.append(nn.Linear(last_dim, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Generate sample paths using an OU process
def generate_sample_path(n_timesteps, mu, theta, sigma):
    x0 = np.random.normal(mu, sigma)
    x = np.zeros(n_timesteps + 1)
    x[0] = x0
    dt = 1 / n_timesteps
    for t in range(1, n_timesteps + 1):
        x[t] = x[t - 1] + theta * (mu - x[t - 1]) * dt + sigma * np.sqrt(dt) * np.random.normal()
    return x

# Compute the truncated log-signature
def compute_log_signature(sample_path, truncation_level_N):
    num_dimensions = len(sample_path[0])
    log_sig = []
    for j in range(len(sample_path)):
        increment = sample_path[j] - sample_path[j - 1] if j > 0 else sample_path[0]
        increment = torch.tensor(increment, dtype=torch.float32)  # Ensure increment is a tensor
        size = int((num_dimensions ** (truncation_level_N + 1) - 1) / (num_dimensions - 1))
        log_sig_n = torch.zeros(size, dtype=torch.float32)
        log_sig_n[0] = 1.0
        if truncation_level_N > 1:
            for n in range(2, truncation_level_N + 1):
                start_idx = int((num_dimensions ** n - 1) / (num_dimensions - 1))
                end_idx = int((num_dimensions ** (n + 1) - 1) / (num_dimensions - 1))
                if start_idx == 0:
                    log_sig_n[1:num_dimensions + 1] = increment
                else:
                    outer_product = torch.outer(log_sig_n[:start_idx], increment).ravel() / n
                    log_sig_n[start_idx:end_idx] = outer_product[:end_idx - start_idx]  # Ensure the slice size matches
        log_sig.append(log_sig_n)
    return torch.stack(log_sig)

# Define the loss function
def loss_fn(net, log_signatures, sample_paths):
    loss = 0
    for m in range(n_samples):
        y0 = sample_paths[m, 0]
        integral = 0
        for j in range(n_timesteps):
            input_tensor = log_signatures[m, j + 1].unsqueeze(0)
            dnn_output = net(input_tensor).item()
            integral += (1 - norm.cdf(integral)) * (sample_paths[m, j + 1] - sample_paths[m, j])
        loss += -(y0 + integral)
    return loss / n_samples

# Generate training data
mu = 0.1
theta = 0.5
sigma = 0.2
sample_paths = torch.tensor([generate_sample_path(n_timesteps, mu, theta, sigma) for _ in range(n_samples)], dtype=torch.float32)
log_signatures = torch.stack([compute_log_signature(sample_path, truncation_level_N) for sample_path in sample_paths])

# Initialize the neural network and optimizer
input_dim = log_signatures.shape[-1]
net = LogSignatureNet(input_dim, hidden_dims)
optimizer = optim.Adam(net.parameters(), lr=lr)

# Train the neural network
for epoch in range(max_epochs):
    optimizer.zero_grad()
    loss = loss_fn(net, log_signatures, sample_paths)
    loss.backward()
    optimizer.step()

# Return the learned policy
def learned_policy(net, test_log_signature):
    with torch.no_grad():
        policy_output = net(test_log_signature.unsqueeze(0)).item()
    return policy_output

# Example usage: Evaluate the learned policy on test data
test_sample_path = torch.tensor(generate_sample_path(n_timesteps, mu, theta, sigma), dtype=torch.float32, device='cuda')
test_log_signature = compute_log_signature(test_sample_path, truncation_level_N)

policy_output = learned_policy(net, test_log_signature)
print(f"Policy output for test data: {policy_output}")


TypeError: len() of a 0-d tensor

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.stats import norm

# Parameters
n_samples = 750
n_timesteps = 75
truncation_level_N = 3
hidden_dims = [32, 64, 32]
max_epochs = 500
lr = 0.001

# Define the neural network architecture
class LogSignatureNet(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super(LogSignatureNet, self).__init__()
        layers = []
        last_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(last_dim, hidden_dim))
            layers.append(nn.ReLU())
            last_dim = hidden_dim
        layers.append(nn.Linear(last_dim, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Generate sample paths using an OU process
def generate_sample_path(n_timesteps, mu, theta, sigma):
    x0 = np.random.normal(mu, sigma)
    x = np.zeros(n_timesteps + 1)
    x[0] = x0
    dt = 1 / n_timesteps
    for t in range(1, n_timesteps + 1):
        x[t] = x[t - 1] + theta * (mu - x[t - 1]) * dt + sigma * np.sqrt(dt) * np.random.normal()
    return x

# Compute the truncated log-signature
def compute_log_signature(sample_path, truncation_level_N):
    sample_path = torch.tensor(sample_path, dtype=torch.float32).clone().detach().requires_grad_(True)  # Ensure sample_path is a tensor
    num_dimensions = 1  # Since sample_path is 1-dimensional
    log_sig = []
    for j in range(sample_path.shape[0]):
        increment = sample_path[j] - sample_path[j - 1] if j > 0 else sample_path[0]
        increment = increment.clone().detach().requires_grad_(True).reshape(-1)  # Ensure increment is a 1D tensor
        if num_dimensions == 1:
            size = truncation_level_N + 1
        else:
            size = int((num_dimensions ** (truncation_level_N + 1) - 1) / (num_dimensions - 1))
        log_sig_n = torch.zeros(size, dtype=torch.float32)
        log_sig_n[0] = 1.0  # Set initial value before requires_grad
        log_sig_n = log_sig_n.clone().detach().requires_grad_(True)
        if truncation_level_N > 1:
            for n in range(2, truncation_level_N + 1):
                if num_dimensions == 1:
                    # Use slicing to create new tensors instead of in-place operations
                    log_sig_n = torch.cat((log_sig_n[:1], increment[:n], log_sig_n[n+1:]))
                else:
                    start_idx = int((num_dimensions ** n - 1) / (num_dimensions - 1))
                    end_idx = int((num_dimensions ** (n + 1) - 1) / (num_dimensions - 1))
                    outer_product = torch.outer(log_sig_n[:start_idx], increment).ravel() / n
                    log_sig_n = torch.cat((log_sig_n[:start_idx], outer_product[:end_idx - start_idx], log_sig_n[end_idx:]))  # Ensure the slice size matches
        log_sig.append(log_sig_n)
    return torch.stack(log_sig)

# Define the loss function
def loss_fn(net, log_signatures, sample_paths):
    loss = 0
    for m in range(n_samples):
        y0 = sample_paths[m, 0]
        integral = 0
        for j in range(n_timesteps):
            input_tensor = log_signatures[m, j + 1].unsqueeze(0)
            dnn_output = net(input_tensor)
            integral += (1 - norm.cdf(integral)) * (sample_paths[m, j + 1] - sample_paths[m, j])
        loss += -(y0 + integral)
    return loss / n_samples

# Generate training data
mu = 0.1
theta = 0.5
sigma = 0.2
sample_paths = np.array([generate_sample_path(n_timesteps, mu, theta, sigma) for _ in range(n_samples)])
sample_paths = torch.tensor(sample_paths, dtype=torch.float32)
log_signatures = torch.stack([compute_log_signature(sample_path, truncation_level_N) for sample_path in sample_paths])

# Initialize the neural network and optimizer
input_dim = log_signatures.shape[-1]
net = LogSignatureNet(input_dim, hidden_dims)
optimizer = optim.Adam(net.parameters(), lr=lr)

# Train the neural network
for epoch in range(max_epochs):
    optimizer.zero_grad()
    loss = loss_fn(net, log_signatures, sample_paths)
    loss.backward()
    optimizer.step()

# Return the learned policy
def learned_policy(net, test_log_signature):
    with torch.no_grad():
        policy_output = net(test_log_signature.unsqueeze(0)).item()
    return policy_output

# Example usage: Evaluate the learned policy on test data
test_sample_path = generate_sample_path(n_timesteps, mu, theta, sigma)
test_log_signature = compute_log_signature(test_sample_path, truncation_level_N)

policy_output = learned_policy(net, test_log_signature)
print(f"Policy output for test data: {policy_output}")



<ipython-input-53-0c846199381d>:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample_path = torch.tensor(sample_path, dtype=torch.float32).clone().detach().requires_grad_(True)  # Ensure sample_path is a tensor


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import norm
import torch
import torch.nn as nn
import torch.optim as optim
import esig.tosig as ts

# Define the neural network
class LogSignatureNet(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super(LogSignatureNet, self).__init__()
        layers = []
        last_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(last_dim, hidden_dim))
            layers.append(nn.ReLU())
            last_dim = hidden_dim
        layers.append(nn.Linear(last_dim, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

def calculate_spread(filename):
    data = pd.read_csv(filename)

    # Check if specified columns exist
    if 'CO1' not in data.columns or 'CL1' not in data.columns:
        raise ValueError(f"Columns 'CO1' and 'CL1' not found in CSV.")

    # Handle missing values by dropping rows with missing data
    data.dropna(subset=['CO1', 'CL1'], inplace=True)  # Drop rows with NaNs in either column

    # Convert pandas Series to NumPy arrays
    d_ts1 = data['CO1'].to_numpy()
    d_ts2 = data['CL1'].to_numpy()

    # Ensure correct shape (no need for adjustments since data has no NaNs)
    d_ts1 = np.diff(d_ts1)
    d_ts2 = np.diff(d_ts2)

    # Compute slope and intercept of the linear regression
    slope, intercept, _, _, _ = stats.linregress(d_ts1, d_ts2)

    # Calculate the spread
    x = d_ts2 - slope * d_ts1

    return x

# Parameters
n_samples = 100 #1000
n_timesteps = 50 #100
truncation_level_N = 3
n_epochs = 10 #0
lr = 0.01
block_size = 10  # Block size for block bootstrapping

# Load data and calculate the spread
filename = 'commodities.csv'
x = calculate_spread(filename)

# Generate sample log-signatures using block bootstrapping
log_signatures = []
for _ in range(n_samples):
    bootstrap_indices = np.random.choice(len(x), size=(n_timesteps,), replace=True)
    bootstrap_data = x[bootstrap_indices]
    signature = ts.stream2sig(bootstrap_data, truncation_level_N)
    log_signatures.append(signature)

log_signatures = np.array(log_signatures)

# Generate dummy payoff process
Y = np.random.randn(n_samples, n_timesteps + 1)

# Define the loss function
def loss_fn(net):
    loss = 0
    for m in range(n_samples):
        y0 = Y[m, 0]
        integral = 0
        for j in range(n_timesteps):
            input_tensor = torch.tensor(log_signatures[m, j], dtype=torch.float32).unsqueeze(0)
            dnn_output = net(input_tensor).item()
            integral += (1 - norm.cdf(integral)) * (Y[m, j + 1] - Y[m, j])
        loss += -(y0 + integral)
    return loss / n_samples

# Optimize the loss function
input_dim = log_signatures.shape[-1]
hidden_dims = [5, 30, 30]  # Example architecture
net = LogSignatureNet(input_dim, hidden_dims)
optimizer = optim.Adam(net.parameters(), lr=lr)
for epoch in range(n_epochs):
    optimizer.zero_grad()
    loss = loss_fn(net)
    loss_tensor = torch.tensor(loss, requires_grad=True)
    loss_tensor.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}")


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import norm
import torch
import torch.nn as nn
import torch.optim as optim
import esig.tosig as ts

# Define the neural network
class LogSignatureNet(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super(LogSignatureNet, self).__init__()
        layers = []
        last_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(last_dim, hidden_dim))
            layers.append(nn.ReLU())
            last_dim = hidden_dim
        layers.append(nn.Linear(last_dim, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

def calculate_spread(filename):
    try:
        data = pd.read_csv(filename)

        # Check if specified columns exist
        if 'CO1' not in data.columns or 'CL1' not in data.columns:
            raise ValueError(f"Columns 'CO1' and 'CL1' not found in CSV.")

        # Handle missing values by dropping rows with missing data
        data.dropna(subset=['CO1', 'CL1'], inplace=True)  # Drop rows with NaNs in either column

        # Convert pandas Series to NumPy arrays
        d_ts1 = data['CO1'].to_numpy()
        d_ts2 = data['CL1'].to_numpy()

        # Ensure correct shape (no need for adjustments since data has no NaNs)
        d_ts1 = np.diff(d_ts1)
        d_ts2 = np.diff(d_ts2)

        # Compute slope and intercept of the linear regression
        slope, intercept, _, _, _ = stats.linregress(d_ts1, d_ts2)

        # Calculate the spread
        x = d_ts2 - slope * d_ts1

        return x
    except Exception as e:
        print(f"Error reading data: {e}")
        return None

# Parameters
n_samples = 500  # Reduced number of samples
n_timesteps = 50  # Reduced number of timesteps
truncation_level_N = 3
n_epochs = 50  # Reduced number of epochs
lr = 0.01
block_size = 5  # Reduced block size

# Load data and calculate the spread
filename = 'commodities.csv'
x = calculate_spread(filename)

if x is not None:
    # Generate sample log-signatures using block bootstrapping
    log_signatures = []
    for _ in range(n_samples):
        bootstrap_indices = np.random.choice(len(x), size=(n_timesteps,), replace=True)
        bootstrap_data = x[bootstrap_indices]
        signature = ts.stream2sig(bootstrap_data, truncation_level_N)
        log_signatures.append(signature)

    log_signatures = np.array(log_signatures)

    # Generate dummy payoff process
    Y = np.random.randn(n_samples, n_timesteps + 1)

    # Define the loss function
    def loss_fn(net):
        loss = 0
        for m in range(n_samples):
            y0 = Y[m, 0]
            integral = 0
            for j in range(n_timesteps):
                input_tensor = torch.tensor(log_signatures[m, j], dtype=torch.float32).unsqueeze(0)
                dnn_output = net(input_tensor).item()
                integral += (1 - norm.cdf(integral)) * (Y[m, j + 1] - Y[m, j])
            loss += -(y0 + integral)
        return loss / n_samples

    # Optimize the loss function
    input_dim = log_signatures.shape[-1]
    hidden_dims = [5, 30, 30]  # Example architecture
    net = LogSignatureNet(input_dim, hidden_dims)
    optimizer = optim.Adam(net.parameters(), lr=lr)
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        loss = loss_fn(net)
        loss_tensor = torch.tensor(loss, requires_grad=True)
        loss_tensor.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.6f}")
else:
    print("Error: Unable to process data. Check file or data format.")


# New Section

In [ ]:
!pip install esig

In [ ]:
!pip install --upgrade numpy

Compute slope, intercept and spread series

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import linregress

# Load the data from the CSV file
data = pd.read_csv('commodities.csv', parse_dates=['Date'])

# Forward fill missing values
data = data.ffill()

# Extract the CO1 and CL1 series
co1 = data['CO1']
cl1 = data['CL1']

# Calculate the differences between the two series
diffs = co1 - cl1

# Remove NaN values from the differences
diffs = diffs.dropna()

# Compute the linear regression
slope, intercept, _, _, _ = linregress(diffs.index, diffs.values)

# Calculate the spread series
spread = diffs - (slope * diffs.index + intercept)

print(f"Slope: {slope}")
print(f"Intercept: {intercept}")
print(f"Spread Series:\n{spread}")


Slope: -0.00041589295632048613
Intercept: 4.631841205045514
Spread Series:
0       4.218159
1       3.908575
2       4.258991
3       3.949406
4       4.309822
          ...   
1304    0.530483
1305    0.440899
1306    0.151315
1307    0.271731
1308    0.252147
Length: 1309, dtype: float64


<ipython-input-108-4c67706da66f>:6: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data = pd.read_csv('commodities.csv', parse_dates=['Date'])


In [ ]:
import pandas as pd

def calculate_spread_series(data):
    """
    Calculates the spread series between CL1 and CO1 based on their linear relationship.

    Args:
        data (pandas.DataFrame): DataFrame containing the CO1 and CL1 columns.

    Returns:
        pandas.Series: The spread series defined by CL1 - [slope * CO1].
    """
    # Forward fill missing values
    data = data.ffill()

    # Extract the CO1 and CL1 series
    co1 = data['CO1']
    cl1 = data['CL1']

    # Calculate the differences between the two series
    diffs_co1 = co1.diff()
    diffs_cl1 = cl1.diff()

    # Remove NaN values from the differences
    diffs_co1 = diffs_co1.dropna()
    diffs_cl1 = diffs_cl1.dropna()

    # Compute the slope of the linear relationship
    slope = diffs_cl1.cov(diffs_co1) / diffs_co1.var()

    # Calculate the spread series
    spread_series = cl1 - slope * co1

    return spread_series

# Load the data from the CSV file
data = pd.read_csv('commodities.csv')

# Calculate the spread series
spread = calculate_spread_series(data)

print("Spread Series:\n", spread)

Spread Series:
 0      -1.178478
1      -0.880174
2      -1.149365
3      -0.905288
4      -1.417337
          ...   
1304    4.298742
1305    4.272845
1306    4.623452
1307    4.399251
1308    4.464972
Length: 1309, dtype: float64


In [ ]:
import numpy as np
from scipy.stats import norm
import torch
import torch.nn as nn
import torch.optim as optim

# Parameters
n_samples = 750
n_timesteps = 75
n_features = 3  # dimensions of the truncated log signatures

# Define Ornstein-Uhlenbeck (OU) process
def ou_process(n_timesteps):
    theta = 0.1
    mu = 0.5
    sigma = 0.3
    dt = 0.1
    x = np.zeros(n_timesteps)
    for t in range(1, n_timesteps):
        x[t] = x[t-1] + theta * (mu - x[t-1]) * dt + sigma * np.sqrt(dt) * np.random.normal()
    return x

# Generate sample paths
sample_paths = np.array([ou_process(n_timesteps) for _ in range(n_samples)])

# Compute augmented truncated log signatures
log_signatures = np.random.randn(n_samples, n_timesteps+1, n_features)  # Just random for demonstration

# Define the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(n_features, 1)

    def forward(self, x):
        return self.fc(x)

# Define the loss function
def loss_fn(net, log_signatures, sample_paths):
    loss = 0
    for m in range(n_samples):
        y0 = sample_paths[m, 0]
        integral = 0
        for j in range(n_timesteps - 1):  # Adjusted range to avoid out-of-bounds index
            input_tensor = torch.tensor(log_signatures[m, j + 1]).unsqueeze(0).float()
            dnn_output = net(input_tensor)
            integral += (1 - norm.cdf(integral)) * (sample_paths[m, j + 1] - sample_paths[m, j])
        loss += -(y0 + integral)
    return torch.tensor(loss / n_samples, requires_grad=True)

# Initialize the neural network
net = Net()

# Define optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Training loop
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    loss = loss_fn(net, log_signatures, sample_paths)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# Once training is done, you can deploy this network against test or live sequential data
# The learned policy is encoded in the weights of the neural network


Epoch 1/100, Loss: -0.14608828588923656
Epoch 2/100, Loss: -0.14608828588923656
Epoch 3/100, Loss: -0.14608828588923656
Epoch 4/100, Loss: -0.14608828588923656
Epoch 5/100, Loss: -0.14608828588923656
Epoch 6/100, Loss: -0.14608828588923656
Epoch 7/100, Loss: -0.14608828588923656
Epoch 8/100, Loss: -0.14608828588923656
Epoch 9/100, Loss: -0.14608828588923656
Epoch 10/100, Loss: -0.14608828588923656
Epoch 11/100, Loss: -0.14608828588923656
Epoch 12/100, Loss: -0.14608828588923656
Epoch 13/100, Loss: -0.14608828588923656
Epoch 14/100, Loss: -0.14608828588923656
Epoch 15/100, Loss: -0.14608828588923656
Epoch 16/100, Loss: -0.14608828588923656
Epoch 17/100, Loss: -0.14608828588923656
Epoch 18/100, Loss: -0.14608828588923656
Epoch 19/100, Loss: -0.14608828588923656
Epoch 20/100, Loss: -0.14608828588923656
Epoch 21/100, Loss: -0.14608828588923656
Epoch 22/100, Loss: -0.14608828588923656
Epoch 23/100, Loss: -0.14608828588923656
Epoch 24/100, Loss: -0.14608828588923656
Epoch 25/100, Loss: -0.14

KeyboardInterrupt: 

# 4. Track the progress/time toward convergence/completion

# 5. Track the profit of the trade

# 6. Plot the enter/exit points against the spread

# 2. Optimize the loss function using NNs

In [ ]:
import numpy as np
from scipy.stats import norm
import torch
import torch.nn as nn
import torch.optim as optim

# Parameters
n_samples = 1000
n_timesteps = 100
truncation_level_N = 3  # Truncation level for log-signature
n_epochs = 1000
lr = 0.01
num_dimensions = 3

# Generate time series data
mu = 0.1
sigma = 0.2
dt = 1 / n_timesteps
X0 = np.zeros((n_samples, 2))

# Discretized OU process
X = np.zeros((n_samples, n_timesteps + 1, 2))
X[:, 0, :] = X0
for i in range(n_samples):
    for t in range(n_timesteps):
        X[i, t + 1, :] = X[i, t, :] + mu * (1 - X[i, t, :]) * dt + sigma * np.sqrt(dt) * np.random.randn(2)

# Payoff process (long first asset, short second asset)
Y = X[:, :, 0] - X[:, :, 1]

# Compute truncated log-signatures
time = np.linspace(0, 1, n_timesteps + 1)
time = np.repeat(time[None, :], n_samples, axis=0)[:, :, None]
augmented_X = np.concatenate((time, X), axis=2)

log_signatures = []
for i in range(n_samples):
    path = augmented_X[i]
    log_sig = []
    for j in range(n_timesteps + 1):
        increment = path[j] - path[j - 1] if j > 0 else path[0]
        num_dimensions = len(increment)

        # Initialize log_sig_n with the correct size
        if truncation_level_N == 1:
            log_sig_n = np.zeros(num_dimensions + 1)
        else:
            size = int((num_dimensions ** (truncation_level_N + 1) - 1) / (num_dimensions - 1))
            log_sig_n = np.zeros(size)
        log_sig_n[0] = 1.0  # Initialize with 1 (empty word)

        if truncation_level_N > 1:
            for n in range(1, truncation_level_N):
                start_idx = int((num_dimensions ** n - 1) / (num_dimensions - 1))
                end_idx = int((num_dimensions ** (n + 1) - 1) / (num_dimensions - 1))
                if n == 1:
                    log_sig_n[start_idx + 1:start_idx + num_dimensions + 1] = increment
                else:
                    outer_product = np.multiply.outer(log_sig_n[:start_idx + 1], increment).ravel() / (n + 1)
                    log_sig_n[start_idx + 1:end_idx + 1] = outer_product[:end_idx - start_idx]
        else:
            log_sig_n[1:num_dimensions + 1] = increment

        log_sig.append(log_sig_n)
    log_signatures.append(np.array(log_sig))
log_signatures = np.array(log_signatures)

# Convert log_signatures to PyTorch tensors
log_signatures = torch.tensor(log_signatures, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)

# Define the neural network
class LogSignatureNet(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super(LogSignatureNet, self).__init__()
        layers = []
        last_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(last_dim, hidden_dim))
            layers.append(nn.ReLU())
            last_dim = hidden_dim
        layers.append(nn.Linear(last_dim, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Define the loss function
def loss_fn(net):
    loss = 0
    for m in range(n_samples):
        y0 = Y[m, 0]
        integral = 0
        for j in range(n_timesteps):
            # Ensure the input tensor requires gradients
            input_tensor = log_signatures[m, j + 1].unsqueeze(0).requires_grad_(True)
            dnn_output = net(input_tensor).item()
            integral += (1 - norm.cdf(integral)) * (Y[m, j + 1] - Y[m, j])
        loss += -(y0 + integral)
    return loss / n_samples

# Optimize the loss function
input_dim = log_signatures.shape[-1]
hidden_dims = [5, 30, 30]  # Example architecture
net = LogSignatureNet(input_dim, hidden_dims)
optimizer = optim.Adam(net.parameters(), lr=lr)
for epoch in range(n_epochs):
    optimizer.zero_grad()
    loss = loss_fn(net)
    # Ensure the loss tensor requires gradients
    loss = torch.tensor(loss, requires_grad=True)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

print(f"Optimal deep signature stopping policy parameters: {list(net.parameters())}")


<ipython-input-27-a5aff9d5efd4>:111: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = torch.tensor(loss, requires_grad=True)


Epoch 0, Loss: 0.003443


KeyboardInterrupt: 